In [1]:
import os
import re
import numpy as np 
import pandas as pd 
import sklearn
import pytesseract
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
conf = r'-- oem 2'

from sklearn.cluster import AgglomerativeClustering

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/keep-babies-safe/dataset/test.csv
/kaggle/input/keep-babies-safe/dataset/images/20f4e86283.jpg
/kaggle/input/keep-babies-safe/dataset/images/9eb2521664.jpg
/kaggle/input/keep-babies-safe/dataset/images/384811545f.jpg
/kaggle/input/keep-babies-safe/dataset/images/2403a85a47.jpg
/kaggle/input/keep-babies-safe/dataset/images/ab4907c89d.jpg
/kaggle/input/keep-babies-safe/dataset/images/81358fabcf.jpg
/kaggle/input/keep-babies-safe/dataset/images/28405116e7.jpg
/kaggle/input/keep-babies-safe/dataset/images/e5cb88939c.jpg
/kaggle/input/keep-babies-safe/dataset/images/0e3790d28d.jpg
/kaggle/input/keep-babies-safe/dataset/images/d54f6e7db0.jpg
/kaggle/input/keep-babies-safe/dataset/images/d40a6a5073.jpg
/kaggle/input/keep-babies-safe/dataset/images/7dc2e40986.jpg
/kaggle/input/keep-babies-safe/dataset/images/283213fad7.jpg
/kaggle/input/keep-babies-safe/dataset/images/114d426fee.jpg
/kaggle/input/keep-babies-safe/dataset/images/7df15294d7.jpg
/kaggle/input/keep-babies-safe/datase

In [4]:
DF = {'photo_n' : [], 'flatten_photo' : [], 'text' : []}

In [5]:
''' ResNet 101 model '''
model = tf.keras.models.Sequential()
model.add(tf.keras.applications.ResNet101(include_top=False, weights='imagenet', pooling='avg',))

171450368/171446536 [==============================] - 8s 0us/step


In [6]:
''' freezing layers '''
model.layers[0].trainable = False

In [7]:
''' defining dataloading Function'''
def data_loading(path, height, width):
    '''reading images '''
    img = cv2.imread(path)
    DF['text'].append(pytesseract.image_to_string(img, config = conf))
    
    ''' resizing image '''
    img = cv2.resize(img, (height, width))
    
    ''' Expanding dimensino '''
    img = img = np.expand_dims(img, 0)

    ''' preprocessing of an image '''
    img = tf.keras.applications.resnet50.preprocess_input(img)
    
    ''' prediction '''
    pred = model.predict(img)
    pred = np.array(pred)
    DF['flatten_photo'].append(pred.flatten())
    

In [8]:
### with this all done lets write the iterrrative loop
def read_images(path):
    list_ = os.listdir(path)
    for mem in list_:
        DF['photo_n'].append(mem)
        img_path = path + '/' + mem
        ''' calling function '''
        data_loading(img_path, 224, 224)

In [9]:
''' path of dataset '''
path = '/kaggle/input/keep-babies-safe/dataset/images'
read_images(path)

In [10]:
training_f_v = np.array(DF['flatten_photo'], dtype='float64')

k_means = AgglomerativeClustering(n_clusters = 2)
k_means.fit(training_f_v)

AgglomerativeClustering()

In [11]:
preds = k_means.labels_
name_pred = []

for m in preds:
    if m == 0:
        name_pred.append('toys')
    else:
        name_pred.append('consumer_products')

In [12]:
text = np.array(DF['text'])
real_text = []


for m in text:
    new_mem = re.sub("\s\s+", " ", m)
    if len(new_mem) == 0 or newMem == " ":
        real_text.append('Unnamed')
        continue
    else:
        real_text.append(str(new_mem))

In [13]:
photo_n = DataFrame['photo_n']
df = {'Image': photo_n, 'Class_of_image': name_pred, 'Brand_name': real_text}

''' creating dataframe '''
df = pd.DataFrame(df)

''' converting to csv '''
df.to_csv('predictions2.csv', index = False)

In [14]:
''' displaying first 50 rows '''
df.head(50)

,Image,Class_of_image,Brand_name
0,20f4e86283.jpg,toys,Unnamed
1,9eb2521664.jpg,consumer_products,come\nbaby\nshampoo\n“SE I '@: HIBISCUS » CHI...
2,384811545f.jpg,toys,
3,2403a85a47.jpg,toys,NI-MH AA 3000mAh 6V\nHs 2020.03.02
4,ab4907c89d.jpg,toys,
5,81358fabcf.jpg,consumer_products,Unnamed
6,28405116e7.jpg,toys,Unnamed
7,e5cb88939c.jpg,toys,Unnamed
8,0e3790d28d.jpg,toys,Unnamed
9,d54f6e7db0.jpg,toys,Unnamed
